# Clase 12/01/2019

## Guía Alumno - @RobotistasMX

### Color spaces

In [1]:
import cv2
print(cv2.__version__)
print([x for x in dir(cv2) if x.startswith('COLOR_')])

3.1.0
['COLOR_BAYER_BG2BGR', 'COLOR_BAYER_BG2BGR_EA', 'COLOR_BAYER_BG2BGR_VNG', 'COLOR_BAYER_BG2GRAY', 'COLOR_BAYER_BG2RGB', 'COLOR_BAYER_BG2RGB_EA', 'COLOR_BAYER_BG2RGB_VNG', 'COLOR_BAYER_GB2BGR', 'COLOR_BAYER_GB2BGR_EA', 'COLOR_BAYER_GB2BGR_VNG', 'COLOR_BAYER_GB2GRAY', 'COLOR_BAYER_GB2RGB', 'COLOR_BAYER_GB2RGB_EA', 'COLOR_BAYER_GB2RGB_VNG', 'COLOR_BAYER_GR2BGR', 'COLOR_BAYER_GR2BGR_EA', 'COLOR_BAYER_GR2BGR_VNG', 'COLOR_BAYER_GR2GRAY', 'COLOR_BAYER_GR2RGB', 'COLOR_BAYER_GR2RGB_EA', 'COLOR_BAYER_GR2RGB_VNG', 'COLOR_BAYER_RG2BGR', 'COLOR_BAYER_RG2BGR_EA', 'COLOR_BAYER_RG2BGR_VNG', 'COLOR_BAYER_RG2GRAY', 'COLOR_BAYER_RG2RGB', 'COLOR_BAYER_RG2RGB_EA', 'COLOR_BAYER_RG2RGB_VNG', 'COLOR_BGR2BGR555', 'COLOR_BGR2BGR565', 'COLOR_BGR2BGRA', 'COLOR_BGR2GRAY', 'COLOR_BGR2HLS', 'COLOR_BGR2HLS_FULL', 'COLOR_BGR2HSV', 'COLOR_BGR2HSV_FULL', 'COLOR_BGR2LAB', 'COLOR_BGR2LUV', 'COLOR_BGR2Lab', 'COLOR_BGR2Luv', 'COLOR_BGR2RGB', 'COLOR_BGR2RGBA', 'COLOR_BGR2XYZ', 'COLOR_BGR2YCR_CB', 'COLOR_BGR2YCrCb', 'COL

In [1]:
import cv2 as cv
import numpy as np
cap = cv.VideoCapture(0)
while(True):
    _ , frame = cap.read()
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    
    cv.imshow('original', frame)
    cv.imshow('H channel', hsv[:,:,0])
    cv.imshow('S channel', hsv[:,:,1])
    cv.imshow('V channel', hsv[:,:,2])
    k = cv.waitKey(5) 
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()

In [1]:
import cv2
import numpy as np
 
cap = cv2.VideoCapture(0)

def nothing(x):
    pass

#Creamos una ventana llamada 'image' en la que habra todos los sliders
cv2.namedWindow('image')
cv2.createTrackbar('Hue Minimo','image',0,255,nothing)
cv2.createTrackbar('Hue Maximo','image',0,255,nothing)
cv2.createTrackbar('Saturation Minimo','image',0,255,nothing)
cv2.createTrackbar('Saturation Maximo','image',0,255,nothing)
cv2.createTrackbar('Value Minimo','image',0,255,nothing)
cv2.createTrackbar('Value Maximo','image',0,255,nothing)
 
while(1):
  _,frame = cap.read() #Leer un frame
  hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) #Convertirlo a espacio de color HSV
 
  #Los valores maximo y minimo de H,S y V se guardan en funcion de la posicion de los sliders
  hMin = cv2.getTrackbarPos('Hue Minimo','image')
  hMax = cv2.getTrackbarPos('Hue Maximo','image')
  sMin = cv2.getTrackbarPos('Saturation Minimo','image')
  sMax = cv2.getTrackbarPos('Saturation Maximo','image')
  vMin = cv2.getTrackbarPos('Value Minimo','image')
  vMax = cv2.getTrackbarPos('Value Maximo','image')
 
  #Se crea un array con las posiciones minimas y maximas
  lower=np.array([hMin,sMin,vMin])
  upper=np.array([hMax,sMax,vMax])
 
  #Deteccion de colores
  mask = cv2.inRange(hsv, lower, upper)
 
  #Mostrar los resultados y salir
  cv2.imshow('camara',frame)
  cv2.imshow('mask',mask)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()

In [ ]:
#Librerias necesarias
import cv2
import numpy as np

maxCirculos=6
maxCirculos=maxCirculos-1

video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    framegrey1 = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    cv2.imshow("Original", framegrey1)
    
    
    gray = cv2.GaussianBlur(framegrey1, (1,1), 2)
    """
    La funcion cv2.medianBlur toma la mediana de todos los pixeles bajo el area del kernel y el elemento central se sustituye por este valor mediano. 
    Esto es muy eficaz contra el ruido en las imagenes.
    Lo interesante es que el elemento central es un valor recien calculado que puede ser un valor de pixel en la imagen o un nuevo valor.
    Pero en el desenfoque mediano, el elemento central siempre es reemplazado por algun valor de pixel en la imagen.
    Reduce el ruido de manera efectiva. Su tamano de nucleo debe ser un numero entero impar positivo.
    """
    
    gray = cv2.medianBlur(gray,5)

    # La funcion transforma la imagen en escala de grises a una imagen binaria
    gray = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,19,3)
    
    #Se crea el kernel que se utilizara para la funcion erode y dilate
    kernel = np.ones((3,3),np.uint8)
    
	  #Erosion quita la estructura de pixeles de la capa mas externa
	  #Dilation agrega pixeles a la estructura de la capa mas externa
    gray = cv2.erode(gray,kernel,iterations = 1) 
    gray = cv2.dilate(gray,kernel,iterations = 1)
    
    #DETECCION DE CIRCULOS
    circles =  cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 10, np.array([]), 100, 25, 5, 100)
    
    #DIBUJAMOS LOS CIRCULOS EN LOS ENCONTRADOS EN FRAME
    #DIBUJARA SOLAMENTE EL MAXIMO DE CIRCULOS PERMITIDO
    circulos=0
    if circles is not None:
        for c in circles[0]:
            cv2.circle(frame, (c[0],c[1]), c[2], (0,255,0),2)
            circulos=circulos+1
            print("Coordenada X: " + str(c[0]))
            print("Coordenada Y: " + str(c[1]))
            if(circulos>maxCirculos):
                break
    else:
        print("NO SE DETECTA EL BALON")
    
  
    cv2.imshow("video", frame)
    cv2.imshow("filtros", gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()

NO SE DETECTA EL BALON
Coordenada X: 110.5
Coordenada Y: 84.5
Coordenada X: 71.5
Coordenada Y: 200.5
Coordenada X: 375.5
Coordenada Y: 88.5
Coordenada X: 161.5
Coordenada Y: 142.5
Coordenada X: 366.5
Coordenada Y: 188.5
Coordenada X: 404.5
Coordenada Y: 226.5
Coordenada X: 115.5
Coordenada Y: 161.5
Coordenada X: 160.5
Coordenada Y: 102.5
Coordenada X: 332.5
Coordenada Y: 87.5
Coordenada X: 162.5
Coordenada Y: 146.5
Coordenada X: 78.5
Coordenada Y: 185.5
Coordenada X: 102.5
Coordenada Y: 199.5
Coordenada X: 222.5
Coordenada Y: 167.5
Coordenada X: 145.5
Coordenada Y: 161.5
Coordenada X: 104.5
Coordenada Y: 102.5
Coordenada X: 87.5
Coordenada Y: 178.5
Coordenada X: 41.5
Coordenada Y: 198.5
Coordenada X: 125.5
Coordenada Y: 108.5
Coordenada X: 94.5
Coordenada Y: 86.5
Coordenada X: 163.5
Coordenada Y: 100.5
Coordenada X: 159.5
Coordenada Y: 140.5
Coordenada X: 78.5
Coordenada Y: 188.5
Coordenada X: 167.5
Coordenada Y: 152.5
Coordenada X: 123.5
Coordenada Y: 168.5
Coordenada X: 366.5
Coorden